In [ ]:
import os
import json


from wand.image import Image as wImg
import pandas as pd
import numpy as np

import torch
from torchvision.transforms import v2
from PIL import Image


import torchvision.models as models


from torch.utils.data import DataLoader

Objetivo desse programa é:


função: ler_svg
-------------------------------------------------------------------------------------------------------------------------------
ler arquivos .svg, salvar o nome e o path para cada arquivo em um data frame

nome do dataframe vai ser: icons-data-base
coluna dos nome dos arquivos .svg: Nome
coluna com os paths dos arquivos .svg: svg path

-------------------------------------------------------------------------------------------------------------------------------

função: converter_svg_para_png
-------------------------------------------------------------------------------------------------------------------------------
transformar os arquivos .svg em .png

através da coluna: svg path, abrirei os arquivos e converterei em .png e salvarei em uma nova pasta, chamada icons-png-database
o nome de cada arquivo .png será escrito de acordo com a coluna: Nome

salvarei o path para os arquivos .png em uma coluna do data frame chamada: png path

-------------------------------------------------------------------------------------------------------------------------------

função: converter_png_em_pil_image
-------------------------------------------------------------------------------------------------------------------------------
converterei os arquivos .png em PIL Image

a função vai ler o path para os arquivos .png através da coluna: png path

salvarei os arquivos PIL Image em uma lista chamada: icons_pil_images

-------------------------------------------------------------------------------------------------------------------------------

função: converter_pil_em_tensor
-------------------------------------------------------------------------------------------------------------------------------

converterei as PIL Image em torch.tensor

a função para converter irá converter as images da lista icons_pil_images

o tensor de cada imagem será salvo em uma lista para ser usada na função analisar_tensor

-------------------------------------------------------------------------------------------------------------------------------

função: analisar_tensor
-------------------------------------------------------------------------------------------------------------------------------

Irei analisar cada tensor para extrair as caracteristicas de cada imagem

os tensores serão dados atraves da coluna do data frame: Tensor

a função ira retornar as caracteristicas de cada imagem em forma de arrays, 
esses arrays serão salvos em uma nova coluna do data frame chamada: Caracteristicas Array

-------------------------------------------------------------------------------------------------------------------------------

tenho que ser capaz de salvar os arrays em um arquivo para depois usa-los em um outro programa

In [ ]:
def ler_dataframe(nome_arquivo_hdf5='icons-database.h5'):

    if os.path.isfile(nome_arquivo_hdf5):

        print(f'\nExiste esse data base no diretório. Lendo-o\n')
        nome_data_base = pd.read_hdf(nome_arquivo_hdf5, key="data")
    
    else:

        print(f'\nAinda não existe um database chamado {nome_arquivo_hdf5} nesse diretório\n')
        jaca = pd.DataFrame()
        return jaca
    
    return nome_data_base

In [ ]:
def ler_svg(path_to_svgs = 'icons-base'):

    db_original = ler_dataframe(nome_arquivo_hdf5='icons-database.h5')

    path_svgs = list()

    for modulo in os.listdir(path_to_svgs):

        for estilo in os.listdir(path_to_svgs+"\\"+modulo):

            path_modulo_estilo = f'{path_to_svgs}\\{modulo}\\{estilo}'

            for icone in os.listdir(path_modulo_estilo):

                path_modulo_estilo_svg = f'{path_to_svgs}\\{modulo}\\{estilo}\\{icone}'
                path_svgs.append(path_modulo_estilo_svg)

    nome_svgs = list()
    modulos = list()
    estilos = list()

    for path in path_svgs:

        nome_icone = path.rsplit(("\\"))[-1]
        nome_icone = nome_icone.replace(".svg" , "")
        nome_svgs.append(nome_icone)

        modulo = path.rsplit(("\\"))[1]
        modulos.append(modulo)
        
        estilo = path.rsplit(("\\"))[2]
        estilos.append(estilo)

    data_base = pd.DataFrame({ 'Nome': nome_svgs, 'Svg Path': path_svgs, 'Modulo': modulos, 'Estilo': estilos})

    if db_original.empty:
        return data_base
    else:
        data_base = data_base[~data_base['Svg Path'].isin(db_original['Svg Path'])]
        return data_base
        

In [ ]:
def converter_svg_para_png(data_base, pasta_dos_pngs='icons-png'):
    
    if not os.path.exists(pasta_dos_pngs):
        os.makedirs(pasta_dos_pngs)
    
    for nome_modulo in data_base['Modulo'].unique():

        if not os.path.exists(pasta_dos_pngs+'\\'+nome_modulo):

            os.makedirs(pasta_dos_pngs+'\\'+nome_modulo)


        for nome_estilo in data_base['Estilo'].unique():

            if not os.path.exists(pasta_dos_pngs+'\\'+nome_modulo+'\\'+nome_estilo):
                os.makedirs(pasta_dos_pngs+'\\'+nome_modulo+'\\'+nome_estilo)


    tensorizar = v2.Compose([
                    v2.Resize((224, 224)),
                    v2.ToImage(), 
                    v2.ToDtype(torch.float32, scale=True),
                    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


    tensores = list()

    icons_pil_image = list()

    png_paths = list()

    for i, row in data_base.iterrows():
        svg_path = row['Svg Path']
        nome = row['Nome']

        nome_arq_png = f"{nome}.png"
        real_png_path = (svg_path.replace('icons-base', 'icons-png'))
        real_png_path = real_png_path.replace(f'\\{nome}.svg', '')
        png_path = os.path.join((real_png_path), nome_arq_png)

        with wImg(filename = svg_path) as img:
            img.format = 'png'
            img.save(filename = png_path)
            png_paths.append(png_path)

            icon = Image.open(png_path).convert('RGB')

            icons_pil_image.append(icon)

            tensor = tensorizar(icon)

            tensores.append(tensor)
        

    data_base['Png Path'] = png_paths
    data_base['PIL Image'] = icons_pil_image


    return data_base, tensores

In [ ]:
def analisar_tensores(data_base, lista_tensores, batch_size = 20):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print(f"Torch device: {device}")

    torch.cuda.empty_cache()

    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    model = torch.nn.Sequential(*(list(model.children())[:-1]))

    model.to(device).eval()


    dataloader = DataLoader(lista_tensores, batch_size = batch_size, shuffle=False)
    
    array_caracteristicas = list()

    for batch in dataloader:
        imgs = batch.to(device) 
        if imgs.dim() == 3: 
            imgs = imgs.unsqueeze(0)

        with torch.no_grad():

            features = model(imgs).squeeze() 
            array_caracteristicas.extend(features.cpu().numpy())

            
    data_base['Array características'] = array_caracteristicas

    data_base['Array características'] = data_base['Array características'].apply(lambda x: json.dumps(x.tolist()))

    return data_base

In [ ]:
def salvar_dataframe(data_base, nome_arquivo_hdf5):

    
    data_base.to_hdf(nome_arquivo_hdf5, key="data", mode="w")

In [ ]:
def main(path_to_svgs = 'icons-base', pasta_dos_pngs = 'icons-png', nome_arquivo_hdf5 = 'icons-database.h5', batch_size = 20):

    data_base = ler_svg(path_to_svgs)

    if data_base.empty:
        print('Nenhum módulo ou estilo novo adicionado a icons-base')
        return None

    data_base, lista_tensores = converter_svg_para_png(data_base, pasta_dos_pngs)

    data_base = analisar_tensores(data_base, lista_tensores, batch_size)

    data_base_original = ler_dataframe(nome_arquivo_hdf5)

    if os.path.isfile(nome_arquivo_hdf5):
        
        data_base = data_base[data_base_original.columns.tolist() + list(set(data_base.columns) - set(data_base_original.columns))]

        data_base = pd.concat([data_base_original, data_base], ignore_index=True)

        salvar_dataframe(data_base, nome_arquivo_hdf5)
        print(f'Arquivo .hdf5 com o database chamado {nome_arquivo_hdf5} atualizado com sucesso\n')

    else:

        salvar_dataframe(data_base, nome_arquivo_hdf5)
        print(f'Arquivo .hdf5 com o database chamado {nome_arquivo_hdf5} criado com sucesso\n')

    print(f"Pasta {pasta_dos_pngs} criada com sucesso\n")
    print('-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n')


In [ ]:
main()